<a href="https://colab.research.google.com/github/PrettyCharity/Bigquery/blob/main/Hacker_News_Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Google Cloud Credentials
import os
from google.cloud import bigquery
# Credentials and the path
credential_path = "key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [2]:
#@title Accessing the hacker_news via bigquery object
client = bigquery.Client()
dataset_ref = client.dataset('hacker_news', project = 'bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [3]:
#@title Exploring the tables of hacker_news
tables = client.list_tables(dataset)
for table in tables:
  print(table.table_id)

comments
full
full_201510
stories


In [4]:
#@title Obtaining 'comments' table
table_ref = dataset_ref.table('comments')
table = client.get_table(table_ref)
preview = client.list_rows(table, max_results = 5).to_dataframe()
print('Table columns:\n\n')
print(preview.columns)

Table columns:


Index(['id', 'by', 'author', 'time', 'time_ts', 'text', 'parent', 'deleted',
       'dead', 'ranking'],
      dtype='object')


In [9]:
#@title Construct the SQL query to prolific commenters
query = """
        SELECT author, COUNT(id) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY author
        HAVING COUNT(id) > 10000
        ORDER BY NumPosts DESC
        """


In [10]:
#@title Fetch data from the server and show the results
# API request - run the query, and convert the results to a pandas DataFrame
query_job = client.query(query)
commenters = query_job.to_dataframe()

commenters.style

,author,NumPosts
0,None,227736
1,tptacek,33839
2,jacquesm,21107
3,jrockway,13626
4,DanBC,12902
5,anigbrowl,11395
6,rayiner,11080
7,sp332,10882
8,davidw,10764
9,dragonwriter,10723
